In [7]:
from __future__ import absolute_import, division, print_function

from os import listdir
import xmltodict
import pandas
from StringIO import StringIO
%matplotlib inline
from tqdm import tqdm
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from praelexis_file_system_provider.data_access_layer import AzureBlobStorageDataAccessLayer
import PyPDF2
import subprocess
import shutil
import os

import pytesseract
from PIL import Image

### Connect to blob storage

In [8]:
blob_dal = AzureBlobStorageDataAccessLayer('devcapitecdata', 'Gt5z0EAAAm5ZETk//WBnRiQUZ9FDB/BmLmbx5ZPRtDwh6vxTDAB/XzLlM+m/epfnDYcZ8JDfMEGg1p9tBW0iFQ==', 'bank-statements', home_directory='drop/Documentum_line_files')

blob_dal.connect()
files = blob_dal.list(path='CI')

print ("Number of files: ", len(files))


Number of files:  8000


In [9]:
print(files[500].name)
print(files[501].name)

0f427a33-cbed-4673-a5d0-79dce1b8d45b.pdf
0f427a33-cbed-4673-a5d0-79dce1b8d45b_POCSTATEMENTSSTD_Capitec__19122017_022834_1154.xml


### Download all pdfs and their xml files

In [10]:
numToTest = len(files)
folderLoc = "./TestAllExamples/"
for i in tqdm(range(numToTest)):
    fileNum = i
    while(files[fileNum].name[-3:]!="pdf"):
        fileNum-=1

    pdfName = files[fileNum].directory + files[fileNum].name
    
    matchString = files[fileNum].name[:-4]
    #print(matchString)
    if matchString in files[fileNum-1].name and fileNum>0:
        xmlName = files[fileNum-1].directory + files[fileNum-1].name
    else:
        xmlName = files[fileNum+1].directory + files[fileNum+1].name
    #print ("Downloading: ", pdfName , " and ", xmlName)
        
    stream = StringIO()
    blob_dal.get(pdfName, stream)
    with open (folderLoc+"Example"+str(i+1)+".pdf", 'w') as tmp:
      stream.seek (0)
      shutil.copyfileobj (stream, tmp)
    
    stream = StringIO()
    blob_dal.get(xmlName, stream)
    with open (folderLoc+"Example"+str(i+1)+".xml", 'w') as tmp:
      stream.seek (0)
      shutil.copyfileobj (stream, tmp)

100%|██████████| 2/2 [00:00<00:00, 10.44it/s]


### Convert and process all pdfs

The pdfs are first converted to tif files. The tif files are then sent through Tesseract. A text file is then generated for each pdf image with the estimated text found using Tesseract.

In [11]:
#import commands
#s=commands.getstatusoutput("tesseract ./TestRandomExamples/Example1.tif ./TestRandomExamples/Example1 -l eng")
#print(s)

In [ ]:
import commands
import time
import glob

%env TESSDATA_PREFIX=/leptonica/tesseract/tessdata

pdfNames = glob.glob(folderLoc+"*.pdf")

print("Number of pdfs found: ",len(pdfNames))

for i in tqdm(range(len(pdfNames))):
    start = time.time()
    tifLoc = str(pdfNames[i][:-4])+".tif"
    s=commands.getstatusoutput("convert -density 300 -units pixelsperinch "+str(pdfNames[i])+" -quality 100 "+tifLoc)
    print(s)
    print("It took ",round(time.time()-start,2)," seconds to convert the pdf.")

    start = time.time()
    command = "tesseract " + tifLoc + " "+str(pdfNames[i][:-4])+" -l eng"
    print("Command: ", command)
    s=commands.getstatusoutput(command)
    print(s)
    print("It took ",round(time.time()-start), " to run tesseract on the tif file.")
    
    print("Deleting the tif file.")
    s=commands.getstatusoutput("rm " + tifLoc)
    print("Tif deleted. Please check.")
    exit(0)

print("Done")

  0%|          | 0/5 [00:00<?, ?it/s]

env: TESSDATA_PREFIX=/leptonica/tesseract/tessdata
Number of pdfs found:  5
(0, '')
It took  11.13  seconds to convert the pdf.
Command:  tesseract ./TestAllExamples/Example1.tif ./TestAllExamples/Example1 -l eng


 20%|██        | 1/5 [02:16<09:07, 136.82s/it]

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6')
It took  126.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted. Please check.
(0, '')
It took  10.53  seconds to convert the pdf.
Command:  tesseract ./TestAllExamples/Example2.tif ./TestAllExamples/Example2 -l eng


 40%|████      | 2/5 [03:35<05:58, 119.46s/it]

...

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2')
It took  28.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted. Please check.
(0, '')
It took  3.72  seconds to convert the pdf.
Command:  tesseract ./TestAllExamples/Example4.tif ./TestAllExamples/Example4 -l eng
(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2')
It took  36.0  to run tesseract on the tif file.
Deleting the tif file.


 80%|████████  | 4/5 [04:50<01:18, 78.07s/it]

Tif deleted. Please check.
(0, '')
It took  14.32  seconds to convert the pdf.
Command:  tesseract ./TestAllExamples/Example5.tif ./TestAllExamples/Example5 -l eng


100%|██████████| 5/5 [11:36<00:00, 176.44s/it]

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6\nPage 7\nPage 8')
It took  392.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted. Please check.
Done
